<br>
<a href="https://www.nvidia.com/en-us/training/">
    <div style="width: 55%; background-color: white; margin-top: 50px;">
    <img src="https://dli-lms.s3.amazonaws.com/assets/general/nvidia-logo.png"
         width="400"
         height="186"
         style="margin: 0px -25px -5px; width: 300px"/>
</a>
<h1 style="line-height: 1.4;"><font color="#76b900"><b>Building Agentic AI Applications with LLMs</h1>
<h2><b>Notebook 3:</b> Using LangGraph For Agentic Loops</h2>
<br>

**Welcome to the third section of the course!**

We now know that we can lock an LLM client into various useful configurations; we can streamline or manually engineer interfaces around our LLM to enable routing, retrieval, software queries, etc. and can tackle challenges with this formulation at various levels of abstraction. We also briefly tried out LangGraph, which seemed to be the more recommended interface for LangChain's agentic support mechanisms. In this follow-on, we will formalize LangGraph around its pros and cons and see how we can leverage it to create a near-arbitrary agent system.

### **Learning Objectives:**
- What is LangGraph and why we should learn how to use it.
- How we can implement interesting agent systems using the LangGraph abstractions. 

<hr><br>

## **Part 1:** Reasoning In A Complex Environment

Recall our ideal agent definition that motivates our decomposition:
- An ideal agent can map any input to any output reasonably well, regardless of complexity or length.

**In the previous notebooks, we established that creating good output is inherently non-trivial with a typical LLM, but some techniques exist to improve the process:**
- **Chain of thought prompting** can help to put the agent on the correct track.
- **Algorithmic execution** can help an LLM solve problems with defined algorithms.
- **Structured output** can help an LLM parameterize to a given structure.

We've now defined how these abilities can be used to help route a system between tools, along paths, and towards desirable configurations. In this section, we're going to investigate the LangChain-recommended mechanisms for implementing such flexible systems, and also try to understand why they might actually be necessary.

----
#### **Tangent: Prerequisite Intuitions**

This course strongly assumes that you've taken some of the previous courses in the series, including possibly [**Building LLM Applications with Prompt Engineering**](https://learn.nvidia.com/courses/course-detail?course_id=course-v1:DLI+C-FX-11+V1), [**Rapid Application Development with LLMs**](https://learn.nvidia.com/courses/course-detail?course_id=course-v1:DLI+S-FX-26+V1), and [**Building RAG Agents with LLMs**](https://learn.nvidia.com/courses/course-detail?course_id=course-v1:DLI+S-FX-15+V1).

**If you've taken these courses, you will know that we *DO NOT NEED* a true agentic framework to have LLM-specified control flow.** All of these systems use some flavor of bespoke systems to create agents that fit with the narrative and allow for control flow modifications.

All of the following scenarios are completely viable mechanisms for implementing control flow with LLMs: 

```python
##########################################################
## Prompt Engineering-Based Fulfillment (in Prompt Eng)
while True:
    prompt = (
        "Chat with the person and be a helpful responding assistant."
        " If they want to stop, output <STOP> at the end of your message.\n\nUSER: " + input("[User]")
    )
    output = llm.invoke(prompt)
    if output.content.strip().endswith("<STOP>"):
        break

##########################################################
## Structured-Output Fulfillment (in RAG course)
llm_routes = {
    "logged_in": struct_llm1,
    "logged_out": struct_llm2,
    "reject_path": reject_llm,
}
llm_chain = llm_routes["logged_out"]
while True:
    prompt = "Fill out the person schema. If found in database, you will see a response. {schema}\n\n{input}"
    output_dict = llm_chain({"schema": schema, "input": input("[User]")})
    if output_dict.get("name") and to_db_query(output_dict):
        llm_chain = llm_routes["logged_in"]
    # ...

##########################################################
## Running-State Chain (in RAG Course)
retrieval_chain = (
    RunnablePassthrough() 
    ## {"input", **}
    | RunnablePassthrough.assign({"retrieval": retriever_chain}) 
    ## -> {"input", "retrieval", **}
    | RunnablePassthrough.assign({"requery": rephrase_chain})    
    ## -> {"input", "retrieval", "requery", **}
    | RunnablePassthrough.assign({"response": prompt | llm | StrOutputParser()})
    ## -> {"input", "retrieval", "requery", "response"}
).invoke({"input" : "hello world!"})
```

If you haven't taken these courses, that's ok! But we will be rolling straight into LangGraph to avoid retreading the same ground. The other courses can be revisited and are all available in a self-paced context.

> **The important thing to note: This is a framework made specifically for productionalizing agents!** It integrates a lot of complexity which may or may not be needed for simple LLM systems, but we will be rolling with it so that you can interface with state-of-the-art solutions by the end of this workshop.

<hr><br>

## **Section 2:** Introducing LangGraph

For this course, we will be introducing **[LangGraph](https://github.com/langchain-ai/langgraph)** as a new addition that allows us to manage the conversation flow using a state graph system. By leveraging LangGraph, we can define the agent's states, transitions, and actions in a structured manner, eliminating the need for a fully-custom event loop. This framework enhances scalability and maintainability, especially when dealing with multi-agent systems or intricate workflows. 

As any framework, it obviously has its marketing material which communicates tons of great feature sets, so we will just provide a link to the homepage for those interested. As the course progresses, you will figure out the key value adds and make up your own mind about its pros and cons - every framework has them, and all frameworks lead with the former.

> <a href="https://langchain-ai.github.io/langgraph/" target="_blank"><img src="images/langgraph-intro.png" style="width: 600px" /></a>
> 
> [**LangGraph Home Page**](https://www.langchain.com/langgraph)

<br>

#### Why Is LangGraph Great Overall?

<details>

- **Because it considers so much**. LangGraph is highly customizable to a fault and forces you to abide by best-practices and constraints that may make no sense when you're starting out... but will become impactful later when you actually try to customize, scale up, productionalize.
- **Because it's already widely-adopted**. There are many examples and off-the-shelf solutions floating around that people can start out with, and quite a few research projects and final deployments are made with it. 
- **Because the techniques are transferable**. If you can really understand LangGraph, the other frameworks and their pros/cons become easier to think about. If you only encounter the simplest framework, all subsequent frameworks will look overly-complicated.

</details>

#### Then Is LangGraph Better Than Custom?

<details>

- **When you know you need the abstractions:** Unlike our while-loop which we could massage into a workable multi-state system, LangGraph takes a state graph approach to modeling the agentic traversal process. As such, it incorporates design patterns which scale naturally to non-sequential and even dynamic routines.
- **When you don't know where to start, but know you want to productionalize:** Unlike our while-loop which we could massage into a workable multi-state system, LangGraph takes a state graph approach to modeling the agentic traversal process. As such, it incorporates design patterns which scale naturally to non-sequential and even dynamic routines.

</details>

#### When Is LangGraph Worse Than Custom?

<details>

- **When you just want to make a simple application with some LLMs and some prompts:** In order to account for various multi-agent-specific feature sets and edge cases, LangGraph implements some strong assumptions which greatly increase its learning curve. If you can implement your solutions in basic LangChain, the runnable paradigm is more than sufficient to streamline your pipeline and bypasses several layers of complexity introduced by LangGraph. If, on the other hand, you know you want to scale your application and can benefit from its well-thought-out features/examples, then perhaps it's worth diving in and getting comfortable.
- **When you need the deepest optimizations:** While LangGraph is amazing, there is still room beyond LangGraph for deeper optimizations and stronger modularization. Those looking to make highly-specialized microservices may be interested in custom multithreading/multiprocessing schemes, advanced graph algorithms, and advanced resource management strategies which LangGraph may not offer.
- **When you just want some persona agents:** You saw the CrewAI API. That one has a much flatter learning curve, but also is not as customizable and is quite tailored for its use-cases. If you would like a lighter approach, that's the pretty simple on-ramp (but will also be easy to backtrack to).

</details>

----

<br>

## **Part 3: Warming Up to the LangGraph Abstraction**

**LangGraph** sits on top of LangChain's lower-level runnables (LCEL) but introduces a stronger concept of **state** and **transitions**. At its core, you define:

1. **A State** (in Python terms, a typed dictionary) that captures the relevant information for your application.  
2. **Nodes**, each a Python function that *reads* from and *updates* that state.  
3. **Edges** that link these nodes in a directed graph—indicating how control should move from one node to another.

This is more than just "build a pipeline." By describing nodes and edges, you're essentially describing how your agent will traverse an environment of possibilities.

#### **Example:** A Simple 2-Node Graph

Below is a compact illustration of what an application implemented with this framework looks like in practice:

In [ ]:
from langchain_nvidia import ChatNVIDIA

llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct", base_url="http://llm_client:9000/v1")

In [ ]:
import uuid
from typing import Optional
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.constants import START, END
from langgraph.graph import StateGraph
from langgraph.types import interrupt, Command
from functools import partial

##################################################################
## Define the authoritative state system (environment) for your use-case

class State(TypedDict):
    """The Graph State for your Agent System"""
    foo: str
    human_value: Optional[str]
    llm_response: Optional[str]
    extra_kwargs: Optional[dict]

##################################################################
## Define the operations (Nodes) that can happen on your environment

def edged_input(state: State):
    """Edge" option where transition is generated at runtime"""
    answer = interrupt("[User]: ")
    print(f"> Received an input from the interrupt: {answer}")
    return {"human_value": answer}           ## <- Edge determined by graph construction (default END)
    # return Command(update={"human_value": answer}, goto="response")

def response(state: State, config=None):
    ## Passing in config will cause connector to stream values to state modification buffer. See graph stream later
    response = llm.invoke(state.get("human_value"), config=config)
    return {"llm_response": response}
    # return Command(update={"llm_response": response}, goto=END)

##################################################################
## Define the system that organizes your nodes (and maybe edges)

builder = StateGraph(State)
builder.add_edge(START, "input")  ## A start node is always necessary
builder.add_node("response", response)
builder.add_node("input", edged_input)
builder.add_edge("input", "response")

##################################################################
## A memory management system to keep track of agent state
checkpointer = MemorySaver()
app = builder.compile(checkpointer=checkpointer)

##################################################################
## A config to define which state you want to use (i.e. states["thread_id"] will be active state pool)
config = {
    "configurable": {
        "thread_id": uuid.uuid4(),
    }
}

app_stream = partial(app.stream, config=config)

#### Breaking Down the Flow

1. **START** → **input**:  
   - The system jumps from the built-in `START` placeholder to the node named `"input"`.  
   - `"input"` is tied to the function `edged_input()`, which *interrupts* to ask the user for input. That user input is stored as `state["human_value"]`.
       - Interrupt literally breaks the control flow and the graph execution ends into a paused state. 

3. **input** → **response**: Next, we go to `"response"`, calling the `response()` function.

    - It takes `state["human_value"]` (the user’s text) and passes it into `llm.invoke(...)`.  
    - The resulting LLM output is stored in `state["llm_response"]`.

5. **response** → **END**: And then, we will go to some last state.
   - We haven’t told LangGraph to proceed anywhere else. Without an additional edge from `"response"` to another node (or back to `"node"`), it will just end. In other words, the control flow halts into a rest state.
   - You can add more edges for more steps, or a loop that returns to `"input"` for multi-turn usage.

#### Actually Running It

There are many ways to run this new chain, and superficially it may look somewhat similar to the runnable stuff we were playing with earlier. However, you'll notice that the interrupt mechanism and state buffers require much more handling. Software engineers who need to deeply integrate this system with their other projects would rejoice from those, but those starting out may admittedly feel a bit lost.

In [ ]:
##################################################################
## Simple Invocation Example

## We can stream over it until an interrupt is received
for chunk in app_stream({"foo": "abc"}):
    print("Chunk Before Command:", repr(chunk))

## If an interrupt is recieved, we can resolve it and continue
if "__interrupt__" in chunk:
    command = Command(resume=input(chunk.get("__interrupt__")[0].value))

##################################################################
## Follow-Up Example (Without streaming is simple case, with streaming is advanced case)

stream_outputs = True

if not stream_outputs:
    ## Stream just the individual outputs from the state-writing buffer
    print("\nSending Command:", repr(command), "\n")
    for chunk in app_stream(command):
        print("\nChunk After Command:", repr(chunk))

else:
    ## Same thing, but actually populate the message stream with streamed responses auto-magically
    seen_metas = dict()
    for chunk, meta in app_stream(command, stream_mode="messages"):
        if meta.get("checkpoint_ns") not in seen_metas:
            print(f"[{meta.get('langgraph_node')}]: ", end="", flush=True)
            # print(f"\nNew Buffer Stream Meta: {meta}\n")
        seen_metas[meta.get("checkpoint_ns")] = meta
        if chunk.content:
            print(chunk.content, end="", flush=True)
        if chunk.response_metadata: # or chunk.usage_metadata: 
            print(f"\n\nChunk with response_metadata: {repr(chunk)}")

In [ ]:
app.get_state(config)

In [ ]:
# list(app.get_state_history(config))

In [ ]:
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

display(Image(app.get_graph().draw_mermaid_png(draw_method=MermaidDrawMethod.API)))

<hr><br>

## **Part 4:** Why Use A Graph Abstraction?

A common argument for why this abstraction is so good comes from the camp of classical dialog modeling folks who may argue that "this is just the correct way to think about dialog." There is an argument to be made here, and you're free to think about it. 

- **Graphs/Networks:** Each **node** is a function that processes or transforms part of your conversation or data. **Edges** specify permissible flows between nodes. For instance, you may want user input first, then an LLM call, then a summarization step, etc.

- **Finite State Machines (FSMs):** In classic FSM terms, each *distinct combination* of state variables and node identity can be interpreted as an FSM "state," and each **edge** is a transition triggered by certain conditions. For instance, the code below transitions from `[User Input Node]` to `[LLM Response Node]` unconditionally. In a richer scenario, you might condition on the user’s text.

- **Markov Chains:** If you’re dealing with stochastic or probability-driven transitions, you can think of each node and subsequent edges as forming a Markov chain. The key difference is that in Markov chains, transitions are typically governed by probability distributions. In LangGraph, you can incorporate your own branching logic—possibly using the LLM’s output to determine next steps.  

> <img src="images/quizbot_state_machine.png" style="width: 1000px;"/>
> 
> <b><a href="https://ai.stanford.edu/blog/quizbot/" target="_blank">Towards an Educational Revolution Through Chatbots (2019)</a></b>

<br>

**This is great and all, but some of you may not be convinced:**
- If you're trained in CS, you'll see that this is just a way of modeling process dependencies with explicit logic. Just like any programming language could.
- The especially critical may also find counter-examples where this abstraction, though sufficient, might be better off transposed such that the states are the nodes and the edges are functions. *Consider what a document mesh or a knowledge graphs might require, and why this transposed setup is interesting.*

So again, why do we need this if we already know how to code in Python? Well, it's actually not because it's trying to hide the coding and conditional logic from you. **It's trying to hide the agentic loop and its productionalization aspects from you.**

<br>

#### **The Agent Loop (For Our Purposes)**

Assume that the agentic decomposition is just a way of breaking down an intractibly-complex global function into local operations:

$$F(X) = \sum_{i=0}^n (e_i \circ f_i \circ d_i)(x_i) \text{ for $i$ agents and local environments } {\bigcup_i x_i} \subseteq X$$

Assume further that this system is especially useful to model system dynamics over time, such that we care about some future state $X_T$ transitioned from some initial state $X_0$ via our repeated application of $F(X)$. Specifically:

$$X_T = X_0 + \sum_{t=0}^{T-1}F(X_t) \text{ where } X_{t+1} = X_t + F(X_t)$$

And then assume that $t$ can be sampled at an arbitrarily-fine resolution, and you have the continuous agent equation! (Feel free to insert the integral in the $[0,T]$ interval, but not necessary). And that's how the real world functions!

In computers, agent-based simulations are also structured like this, and there exists a discrete time loop at some level of abstraction. The issues are:
- There are a discrete number of processes, and processes invoke other processes.
- For a given time-step, the number of processes can be very large or very small. Usually, this can can only be determined by observation.
- We like to observe, monitor, control, and version (time-travel), and this requires memory and compute overhead and hard design decisions.
- The number of processes can scale dramatically with the number of users, complexity of macro-processes, and increase or resources.

And we have reconstructed the problem of process management in computer architectures. Which we generally don't have to worry about. So why bother now?

**Because now we're doing it in an high-abstraction framework like Python where discrete-time simulation is very inefficient and hard to scale manually, and need the flexibility to define our own processes propagation, monitoring, concurrent execution, replication, and versioning.** And it turns out that's a challenge, as discussed in the lecture.

<br>

#### **Does that mean LangGraph is the Answer?**

LangGraph is AN answer, and has a relatively small barrier to entry given the level of customization it supports. Therefore, we will continue to use it as our chief abstraction for this course. It still has some clear limitations associated with it, and technically there are higher barrier-of-entry options that have less of those:
- **Custom graph systems** can be implemented using some key abstractions, and they can be truly customized for any arbitrary use-cases. This is, in fact, what LangGraph creates and supports under the hood. However, this problem explodes in complexity as you start implementing the feature sets necessary in modern-day LLM applications, and your solutions will be inherently bespoke unless released as a unified framework.
- [**NVIDIA Morpheus**](https://www.nvidia.com/en-us/ai-data-science/products/morpheus/) is another useful abstraction which offers advanced data pipeline solutions that could be used to pipe inference streams, track analytics, and optimize pipelines with CUDA-accelerated workflows. With that being said, it doesn't have all of the niceties associated with LangGraph for the agentic use-case. The Morpheus - LangGraph divide is similar to the LangGraph - CrewAI divide in terms of complexity, so it's not actually a bad idea to use, but will have less built-in conveniences.

<hr><br>

## **Part 5:** [Exercise] Adding Simple Routing

Now that we're committed to our framework, let's conform to the framework's expectations and implement its signature feature; **routing**. In the immediate next notebook, we will have a more involved example which will show how to integrate the LangGraph event loop with structured output to implement our teased-upon persona agent system from Section 1. Before then, we wanted to include a simple exercise to augment the loop *just enough* to construct a stop condition. 

- The method `get_nth_message` is provided to get the last (or maybe some other) message from the state that is passed into it.
- Using this, try to force the loop to end when the user says "stop". Or maybe when the LLM says `stop`? One or the other is ok.
- The streaming logic has gotten more complicated and is now a generator than handles more cases (including other state buffer streams and the debug buffer). It is implemented in `stream_from_app_1`, and you should try to use it instead.
    - You can also import `stream_from_app` from `course_utils.py` (or transpose it as `from course_utils import stream_from_app`). This one is a bit better and interfaces with a web service! 

In [ ]:
import uuid
from typing import Annotated, Optional
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.constants import START, END
from langgraph.graph import StateGraph
from langgraph.types import interrupt, Command
from langgraph.graph.message import add_messages
from functools import partial
from colorama import Fore, Style
from copy import deepcopy
import operator

##################################################################
## Define the authoritative state system (environment) for your use-case

class State(TypedDict):
    """The Graph State for your Agent System"""
    messages: Annotated[list, add_messages]
    interactions: Annotated[int, operator.__add__]
    extra_kwargs: Optional[dict]

def get_nth_message(state: State, n=-1, attr="messages"):
    try: return state.get("messages")[n].content
    except: return ""
    
##################################################################
## Define the operations (Nodes) that can happen on your environment

def user(state: State):
    """Edge" option where transition is generated at runtime"""
    answer = interrupt("[User]:")
    return {"messages": [("user", answer)]} 

def agent(state: State, config=None):
    ## Passing in config will cause connector to stream values to state modification buffer. See graph stream later
    response = llm.invoke(state.get("messages"), config=config)
    return {"messages": [response]}

def route(state: State, config=None):
    ## TODO: In the case of "stop" being found in the current state,
    ## go to the end. Otherwise, route back to the user.
    return {"interactions": 1}

##################################################################
## Define the system that organizes your nodes (and maybe edges)

builder = StateGraph(State)
builder.add_edge(START, "user")  ## A start node is always necessary
builder.add_node("agent", agent)
builder.add_node("user", user)
builder.add_node("route", route)    ## Route node declaration
builder.add_edge("user", "agent")
builder.add_edge("agent", "route")  ## Route edge declaration

##################################################################
## A memory management system to keep track of agent state
checkpointer = MemorySaver()
app = builder.compile(checkpointer=checkpointer)
config = {"configurable": {"thread_id": uuid.uuid4()}}
app_stream = partial(app.stream, config=config)

##################################################################
## Simple Invocation Example

def stream_from_app_simple(app_stream, input_buffer=[{"messages": []}], verbose=False, debug=False):
    """Executes the agent system in a streaming fashion."""
    seen_metas = dict()
    input_buffer = deepcopy(input_buffer)
    
    while input_buffer:
        for mode, chunk in app_stream(input_buffer.pop(), stream_mode=["values", "messages", "updates", "debug"]):
            if mode == "messages":
                chunk, meta = chunk
                if meta.get("checkpoint_ns") not in seen_metas:
                    caller_node = meta.get("langgraph_node")
                    yield f"[{caller_node.title()}]: "
                seen_metas[meta.get("checkpoint_ns")] = meta
                if chunk.content:
                    yield chunk.content
            elif mode == "values" and verbose:
                print("[value]", chunk, flush=True)
            elif mode == "updates":
                if verbose: 
                    print("[update]", chunk, flush=True)
                if "__interrupt__" in chunk and chunk.get("__interrupt__")[0].resumable:
                    user_input = input("\n[Interrupt] " + chunk.get("__interrupt__")[0].value)
                    input_buffer.append(Command(resume=user_input))
            elif mode == "debug" and debug:
                print(f"[debug] {chunk}", flush=True)

# from course_utils import stream_from_app

import time

## We can stream over it until an interrupt is received
for token in stream_from_app_simple(app_stream, verbose=False, debug=False):
    print(token, end="", flush=True)

<br>

**NOTE: If you happen to be using the `stream_from_app` from `course_utils`, the following interface should be accessible:**

In [ ]:
%%js
var url = 'http://'+window.location.host+':3002';
element.innerHTML = '<a style="color:#76b900;" target="_blank" href='+url+'><h2>< Link To Trace Frontend ></h2></a>';

<br><details><summary><b>Solution</b></summary>

```python

def route(state: State, config=None):
    ## TODO: In the case of "stop" being found in the current state,
    ## go to the end. Otherwise, route back to the user.
    if "stop" in get_nth_message(state, n=-2): 
        return {"interactions": 1}  ## Implied goto=END
    return Command(update={"interactions": 1}, goto="user")

```

</details>

<hr><br>

## **Part 5:** Reflecting On This Exercise

For those of you just starting out with building agent systems of any kind, LangGraph may seem intimidating. In fact, many engineers who have more limited-scope problems can probably get by with just the LangChain abstractions from the previous sections (which the previous courses go into depth on). At the same time, more experienced engineers who have a strong grasp of agentic software paradigms can straddle the line anywhere between primitive-based and framework-enabled orchestration, and can hopefully make the right design decisions to enable their solution at any scale. 

We like to treat LangGraph as a great starting abstraction which works **by default** for some of the largest models, **with extra effort** for a selection of models, and **with significant modification** for an even-greater set of possible interfaces. *(Unfortunately, the Llama-8B model may fall under the middle or last category, as you will see in the next notebook)*. With that said, at least it has some simple ways into the ecosystem and can scale by default into production use-cases without too much hassle. As such, we will use this framework for the rest of the course as necessary to enable our agentic loops.

- In the next **Exercise Notebook**, we will leverage LangGraph to recreate our multi-persona abstraction with next-speaker selection and a custom state system to show that the framework is *at least* flexible enough to create near-arbitrary state interfaces.

<a href="https://www.nvidia.com/en-us/training/">
    <div style="width: 55%; background-color: white; margin-top: 50px;">
    <img src="https://dli-lms.s3.amazonaws.com/assets/general/nvidia-logo.png"
         width="400"
         height="186"
         style="margin: 0px -25px -5px; width: 300px"/>